In [39]:
!pip freeze | grep scikit-learn

In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [42]:
year = 2022
month = 3

In [43]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')


KeyboardInterrupt



In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [ ]:
y_pred.std()

In [ ]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [ ]:
df['predictions'] = y_pred

In [ ]:
df_result = df[['ride_id', 'predictions']]

In [33]:
output_file = "./saved_predictions.parquet"

In [34]:
df_result

,ride_id,predictions
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904
...,...,...
2979426,2022/02_2979426,12.038225
2979427,2022/02_2979427,11.441569
2979428,2022/02_2979428,11.890459
2979429,2022/02_2979429,15.102681


In [35]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)